<a href="https://colab.research.google.com/github/Xuehui-Jiang/Uni-Mannheim-Master-Thesis/blob/main/Experimental_Evaluation/Experiment_Albums.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cloning the pyRDF2Vec repo
!git clone https://github.com/IBCNServices/pyRDF2Vec.git

Cloning into 'pyRDF2Vec'...
remote: Enumerating objects: 7425, done.
remote: Counting objects: 100% (393/393), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 7425 (delta 242), reused 261 (delta 137), pack-reused 7032
Receiving objects: 100% (7425/7425), 5.35 MiB | 20.66 MiB/s, done.
Resolving deltas: 100% (5043/5043), done.


In [ ]:
!pip uninstall community
!pip install python-louvain
!pip install pyRDF2Vec --upgrade

Found existing installation: community 1.0.0b1
Uninstalling community-1.0.0b1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/community-1.0.0b1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/community/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/community/community_louvain.py
    /usr/local/lib/python3.10/dist-packages/community/community_status.py
Proceed (Y/n)? Y
  Successfully uninstalled community-1.0.0b1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install rdflib
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')
import functools
import itertools
from typing import List, Sequence, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rdflib
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC

import sys
sys.path.append('pyRDF2Vec')
sys.path.append('pyRDF2Vec/pyrdf2vec')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.walkers import RandomWalker

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Saving MetacriticAlbums.tsv to MetacriticAlbums.tsv
User uploaded file "MetacriticAlbums.tsv" with length 510740 bytes


In [ ]:
# Load data
complete_df = pd.read_csv('MetacriticAlbums.tsv', sep='\t')

# Remove rows with duplicate 'DBpedia_URI'
complete_df.drop_duplicates(subset=['DBpedia_URI'], keep=False, inplace=True)

entities = complete_df['DBpedia_URI'].tolist()
labels = complete_df['label'].tolist()
reg_targets = complete_df['rating'].tolist()

In [ ]:
print(len(entities))

1584


In [ ]:
# Filter out entities and relationships directly related to Albums Dataset
with open("/content/drive/My Drive/dbpedia_files/direct_merged_dbpedia.ttl", "r") as file, open("direct_albums_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2:
            # Check if the subject or object of the triple is in our album URL list
            if triple[0].strip('<>') in entities or triple[2].strip('<>') in entities:
                out_file.write(line)

In [ ]:
# Filter out entities and relationships directly or indirectly related to Cities Dataset
entities_related_to_albums = set(entities)
new_entities = set(entities)
iteration_count = 0

while new_entities and iteration_count < 2:
    temp_entities = set()
    with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file:
        for line in file:
            triple = line.strip().split()
            if len(triple) > 2 and (triple[0].strip('<>') in new_entities or triple[2].strip('<>') in new_entities):
                    temp_entities.add(triple[0].strip('<>'))
                    temp_entities.add(triple[2].strip('<>'))

    new_entities = temp_entities - entities_related_to_albums
    entities_related_to_albums.update(new_entities)
    iteration_count += 1

# Finally, we can write all triples that are directly or indirectly related to entities in the `entities_related_to_cities` collection
with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file, open("indirect_albums_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2 and (triple[0].strip('<>') in entities_related_to_albums or triple[2].strip('<>') in entities_related_to_albums):
            out_file.write(line)


In [ ]:
# Define the list of files
files_to_merge = [
    "direct_albums_filtered_dbpedia.ttl",
    "indirect_albums_filtered_dbpedia.ttl"
]

# Merge files
with open("albums_filtered_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open(file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
input_file = "albums_filtered_dbpedia.ttl"
output_file = "cleaned_albums_filtered_dbpedia.ttl"

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Determine whether it is a comment or a blank line
        if line.startswith("#") or line.strip() == "":
            continue

        # Split triples
        triple = line.strip().split()

        # Make sure this is a complete triplet
        if len(triple) > 2:
            # Check whether the object starts with "http://dbpedia.org/resource/"
            if triple[2].startswith("<http://dbpedia.org/resource/"):
                outfile.write(line)

print(f"Finished filtering {input_file} and saved the result to {output_file}")


Finished filtering albums_filtered_dbpedia.ttl and saved the result to cleaned_albums_filtered_dbpedia.ttl


In [ ]:
!head -n 10 cleaned_albums_filtered_dbpedia.ttl

<http://dbpedia.org/resource/Songs_for_the_Deaf> <http://dbpedia.org/property/studio> <http://dbpedia.org/resource/Conway_Recording_Studios> .
<http://dbpedia.org/resource/Songs_for_the_Deaf> <http://dbpedia.org/property/rev> <http://dbpedia.org/resource/AllMusic> .
<http://dbpedia.org/resource/Songs_for_the_Deaf> <http://dbpedia.org/property/rev> <http://dbpedia.org/resource/Pitchfork_Media> .
<http://dbpedia.org/resource/Songs_for_the_Deaf> <http://dbpedia.org/property/writer> <http://dbpedia.org/resource/Josh_Homme> .
<http://dbpedia.org/resource/Songs_for_the_Deaf> <http://dbpedia.org/property/writer> <http://dbpedia.org/resource/Mario_Lalli> .
<http://dbpedia.org/resource/Songs_for_the_Deaf> <http://dbpedia.org/property/extra> <http://dbpedia.org/resource/Nick_Oliveri> .
<http://dbpedia.org/resource/Songs_for_the_Deaf> <http://dbpedia.org/property/title> <http://dbpedia.org/resource/No_One_Knows> .
<http://dbpedia.org/resource/Songs_for_the_Deaf> <http://dbpedia.org/property/title

In [ ]:
import shutil

destination_path = "/content/drive/My Drive/dbpedia_files"

shutil.copy("cleaned_albums_filtered_dbpedia.ttl", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("/content/drive/My Drive/dbpedia_files/cleaned_albums_filtered_dbpedia.ttl", fmt='turtle')

kgentities = kg._entities

In [ ]:
for e in entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

http://dbpedia.org/resource/Illinois_(album)
http://dbpedia.org/resource/Hell_Hath_No_Fury
http://dbpedia.org/resource/Jj_n%C2%B0_2
http://dbpedia.org/resource/Life..._The_Best_Game_in_Town
http://dbpedia.org/resource/Second_Chance_(album)
http://dbpedia.org/resource/Sugar_Mountain_%E2%80%93_Live_at_Canterbury_House_1968
http://dbpedia.org/resource/Quantic_&_Alice_Russell:_Look_Around_The_Corner
http://dbpedia.org/resource/03/07%E2%80%9309/07
http://dbpedia.org/resource/Hold_on_Now,_Youngster
http://dbpedia.org/resource/The_Evangelist
http://dbpedia.org/resource/Life_Cycles_(album)
http://dbpedia.org/resource/Time_to_Die
http://dbpedia.org/resource/Body_Language_Live
http://dbpedia.org/resource/In_Some_Way,_Shape_or_Form
http://dbpedia.org/resource/The_Oracle_(album)
http://dbpedia.org/resource/Ariels
http://dbpedia.org/resource/Legion_of_Boom
http://dbpedia.org/resource/Mr._A%E2%80%93Z
http://dbpedia.org/resource/What_Is_Love%3F_(album)
http://dbpedia.org/resource/Wild_Ones_(album)


In [ ]:
# Entities not in KG
absent_entities = [
    "http://dbpedia.org/resource/Illinois_(album)",
    "http://dbpedia.org/resource/Hell_Hath_No_Fury",
    "http://dbpedia.org/resource/Jj_n%C2%B0_2",
    "http://dbpedia.org/resource/Life..._The_Best_Game_in_Town",
    "http://dbpedia.org/resource/Second_Chance_(album)",
    "http://dbpedia.org/resource/Sugar_Mountain_%E2%80%93_Live_at_Canterbury_House_1968",
    "http://dbpedia.org/resource/Quantic_&_Alice_Russell:_Look_Around_The_Corner",
    "http://dbpedia.org/resource/03/07%E2%80%9309/07",
    "http://dbpedia.org/resource/Hold_on_Now,_Youngster",
    "http://dbpedia.org/resource/The_Evangelist",
    "http://dbpedia.org/resource/Life_Cycles_(album)",
    "http://dbpedia.org/resource/Time_to_Die",
    "http://dbpedia.org/resource/Body_Language_Live",
    "http://dbpedia.org/resource/In_Some_Way,_Shape_or_Form",
    "http://dbpedia.org/resource/The_Oracle_(album)",
    "http://dbpedia.org/resource/Ariels",
    "http://dbpedia.org/resource/Legion_of_Boom",
    "http://dbpedia.org/resource/Mr._A%E2%80%93Z",
    "http://dbpedia.org/resource/What_Is_Love%3F_(album)",
    "http://dbpedia.org/resource/Wild_Ones_(album)"
]

# Filter out absent entities
filtered_entities = [e for e in entities if e not in absent_entities]
filtered_labels = [label for entity, label in zip(entities, labels) if entity not in absent_entities]
filtered_reg_targets = [reg_targets for entity, reg_targets in zip(entities, reg_targets) if entity not in absent_entities]

In [ ]:
print(len(filtered_entities))

1564


In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

Prepare for Knowledge Graph

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
import networkx as nx

In [ ]:
# Load the graph from the input file
graph = Graph()
graph.parse("/content/drive/My Drive/dbpedia_files/cleaned_albums_filtered_dbpedia.ttl", format="turtle")

<Graph identifier=Na27422591d364b6ca42e8d9f005e7583 (<class 'rdflib.graph.Graph'>)>

In [ ]:
G = nx.DiGraph()  # Create a directed graph

for s, p, o in graph:
    G.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G.nodes())
num_edges = len(G.edges())
print("The DBpedia knowledge graph as of 2016-10 has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The DBpedia knowledge graph as of 2016-10 has 2663803 nodes and 7440404 edges.


Classification & Regression tasks based graph embeddings without SA

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, random_state=42)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, workers=1, sample=0, ns_exponent=0.75, epochs=5)

        # Initialize an empty list to store embeddings
        all_embeddings = []

        # Define the batch ranges
        batches = [(812, 825), (825, 850), (850, 900), (900, 1000), (1000, 1190), (1190, 1385), (1385, 1482), (1482, 1581)]

        # Loop through each batch range
        for start, end in batches:
            batch = filtered_entities[start:end]
            print(f"Processing batch {start}-{end} with {len(batch)} entities...")

            # Initialize a new transformer for each batch
            transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

            try:
                embeddings, _ = transformer.fit_transform(kg, batch)
                all_embeddings.extend(embeddings)
            except Exception as e:
                print(f"An error occurred while processing batch {start}-{end}: {e}")

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=3)
        kf = KFold(n_splits=3)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            #'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels[1482:1581], cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets[1482:1581], cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('ttl_class_reg_results.csv')

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('pure_class_reg_results.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.680316
1           cbow         50                 C4.5  Accuracy   0.603520
2           cbow         50            KNN (k=3)  Accuracy   0.723812
3           cbow         50                  SVM  Accuracy   0.751262
4           cbow         50    Linear Regression      RMSE  14.847997


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with BFS

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving main.py to main.py
Saving sa_helper.py to sa_helper.py


In [ ]:
from main import *
from sa_helper import *

In [ ]:
# Count the neighbor node number for each nodes in the graph
neighbor_counts = {node: len(list(G.neighbors(node))) for node in G.nodes()}

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub.nodes())
num_edges = len(G_sub.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 14909 nodes and 34314 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.712322
1           cbow         50                 C4.5  Accuracy   0.755206
2           cbow         50            KNN (k=3)  Accuracy   0.873502
3           cbow         50                  SVM  Accuracy   0.872807
4           cbow         50    Linear Regression      RMSE  13.794288


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 14909 nodes and 34314 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from google.colab import files
files.download('BFS_subgraph_tf.nt')

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.682288
1           cbow         50                 C4.5  Accuracy   0.746795
2           cbow         50            KNN (k=3)  Accuracy   0.847893
3           cbow         50                  SVM  Accuracy   0.856765
4           cbow         50    Linear Regression      RMSE  12.390754


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 14909 nodes and 34314 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from google.colab import files
files.download('BFS_subgraph_ft.nt')

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.684881
1           cbow         50                 C4.5  Accuracy   0.727621
2           cbow         50            KNN (k=3)  Accuracy   0.857509
3           cbow         50                  SVM  Accuracy   0.831239
4           cbow         50    Linear Regression      RMSE  15.275252


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 14909 nodes and 34314 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from google.colab import files
files.download('BFS_subgraph_tt.nt')

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.764196
1           cbow         50                 C4.5  Accuracy   0.758313
2           cbow         50            KNN (k=3)  Accuracy   0.854340
3           cbow         50                  SVM  Accuracy   0.877878
4           cbow         50    Linear Regression      RMSE  12.154972


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with DFS

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub_ff = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub_ff.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ff.nodes())
num_edges = len(G_sub_ff.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 14908 nodes and 34314 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from google.colab import files
files.download('DFS_subgraph_ff.nt')

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.686061
1           cbow         50                 C4.5  Accuracy   0.702772
2           cbow         50            KNN (k=3)  Accuracy   0.876731
3           cbow         50                  SVM  Accuracy   0.845999
4           cbow         50    Linear Regression      RMSE  13.227293


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.1, beta=0.009, max_depth=2, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 14908 nodes and 34314 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.656663
1           cbow         50                 C4.5  Accuracy   0.741083
2           cbow         50            KNN (k=3)  Accuracy   0.850474
3           cbow         50                  SVM  Accuracy   0.850441
4           cbow         50    Linear Regression      RMSE  13.958708


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 14908 nodes and 34314 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
from google.colab import files
files.download('DFS_subgraph_ft.nt')

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.684799
1           cbow         50                 C4.5  Accuracy   0.716156
2           cbow         50            KNN (k=3)  Accuracy   0.842814
3           cbow         50                  SVM  Accuracy   0.858117
4           cbow         50    Linear Regression      RMSE  13.514335


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 14908 nodes and 34314 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.729528
1           cbow         50                 C4.5  Accuracy   0.741691
2           cbow         50            KNN (k=3)  Accuracy   0.844721
3           cbow         50                  SVM  Accuracy   0.854205
4           cbow         50    Linear Regression      RMSE  12.682367
